In [3]:
from pathlib import Path

In [4]:
thumbnail_dir = Path("/home/dani/dev/conny-dev/image-classification/data/input/thumbnails/")

In [5]:
hashes = pd.read_csv("/home/dani/dev/conny-dev/image-classification/notebooks/hashed_metadata.csv")
hashes = hashes.rename(columns={hashes.columns[0]:"thumbnail_path", "path":"file_name"})
hashes["hash"] = hashes["thumbnail_path"].apply(lambda string: string.split(".")[0])
hashes["thumbnail_path"] = hashes["thumbnail_path"].apply(lambda string: str((thumbnail_dir / string).absolute()))

I seek:
- write a prepprocessing script that reduces image sizes via thumbnails
    - saving a csv with (`original_path`,`hash`, `thumbnail_path`)
- then our labeling script should operate on the thumbnail directory
- it takes very long to build that list, so lets re-create it using the data we have so far

In [53]:
from labeling.preprocess import _load_image, _crop, _make_thumbnail, _hash

In [43]:
i = _load_image(hashes["file_name"].values[100])

In [56]:
t = _make_thumbnail(i)

In [55]:
_hash(t)

'506cd8b17789daf1f2872e03f18d73699b58d94257416ae97924ec9b9ba6d006'

In [8]:
metadata = hashes.dropna()[["thumbnail_path", "label"]]
metadata = metadata.rename(columns={"thumbnail_path":"file_name"})
metadata

,file_name,label
110,/home/dani/dev/conny-dev/image-classification/...,world
112,/home/dani/dev/conny-dev/image-classification/...,document
148,/home/dani/dev/conny-dev/image-classification/...,document
155,/home/dani/dev/conny-dev/image-classification/...,world
242,/home/dani/dev/conny-dev/image-classification/...,world
...,...,...
4782,/home/dani/dev/conny-dev/image-classification/...,world
4798,/home/dani/dev/conny-dev/image-classification/...,world
4818,/home/dani/dev/conny-dev/image-classification/...,world
4936,/home/dani/dev/conny-dev/image-classification/...,world


In [10]:
metadata.to_json("/home/dani/dev/conny-dev/image-classification/data/output/metadata.jsonl", orient="records", lines=True)

In [11]:
hashes = hashes[["file_name", "hash", "thumbnail_path"]]

In [12]:
hashes

,file_name,hash,thumbnail_path
0,/home/dani/dev/conny-dev/image-classification/...,5bc4e989f6cd5e0bcd0068fedf70d91d206b72c02ad9a7...,/home/dani/dev/conny-dev/image-classification/...
1,/home/dani/dev/conny-dev/image-classification/...,b305af264ce943bbed3074ef363e083a9329498c4b71ff...,/home/dani/dev/conny-dev/image-classification/...
2,/home/dani/dev/conny-dev/image-classification/...,a2cbda1cf67393c519bc520bf7358f71257d5e28488e95...,/home/dani/dev/conny-dev/image-classification/...
3,/home/dani/dev/conny-dev/image-classification/...,60e700d1f2fd9b6fa469fb21ae3bd56cf85bb3771575f2...,/home/dani/dev/conny-dev/image-classification/...
4,/home/dani/dev/conny-dev/image-classification/...,a7f0594d7838c88c7fdf253ebb15d2d8c9f1652ec68bc5...,/home/dani/dev/conny-dev/image-classification/...
...,...,...,...
24127,/home/dani/dev/conny-dev/image-classification/...,619b9256fe92ac4038e998b4e8dbc0872843824859316d...,/home/dani/dev/conny-dev/image-classification/...
24128,/home/dani/dev/conny-dev/image-classification/...,6dbdc2bab0c7327910e5aea8ad4c16dcaa5bfa6d2baa0d...,/home/dani/dev/conny-dev/image-classification/...
24129,/home/dani/dev/conny-dev/image-classification/...,30e59b7ecc395a2d78c5babb7225ecc3f497d1757ba25e...,/home/dani/dev/conny-dev/image-classification/...
24130,/home/dani/dev/conny-dev/image-classification/...,002f8ea15bb93d3c18b3ca610c0e11d3624d91eabdc317...,/home/dani/dev/conny-dev/image-classification/...


In [13]:
hashes.to_csv(thumbnail_dir/"hashes.csv", index=False)

In [15]:
from pathlib import Path

In [16]:
from datasets import load_dataset, Image, Features, ClassLabel, Dataset

In [59]:
import PIL

In [60]:
PIL.Image

<module 'PIL.Image' from '/home/dani/miniconda3/envs/conny_ml/lib/python3.9/site-packages/PIL/Image.py'>

In [17]:
img_dir = "/home/dani/dev/conny-dev/image-classification/data/input/thumbnails/"
metadata_path = "/home/dani/dev/conny-dev/image-classification/data/output/metadata.jsonl"
labels = ["world", "document", "SKIP"]

In [18]:
import typing

In [19]:
from labeling.utils import load_dataset_from_disk, is_labeled, is_not_skipped, to_dataset, _dataset_to_list

In [28]:
%%time
img_dir = Path(img_dir)
dataset = load_dataset_from_disk(img_dir, metadata_path, labels)

Resolving data files:   0%|          | 0/5089 [00:00<?, ?it/s]

Using custom data configuration default-734409df78c3d7ce
Found cached dataset imagefolder (/home/dani/.cache/huggingface/datasets/imagefolder/default-734409df78c3d7ce/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


CPU times: user 5.4 s, sys: 1.28 s, total: 6.68 s
Wall time: 6.99 s


In [29]:
[d for d in dataset if d["label"] is not None]

[]

In [30]:
from labeling.samplers import ActiveLearningSampler

In [31]:
from labeling.annotator import Annotator

In [32]:
sampler = ActiveLearningSampler(labels=labels)

In [13]:
annotator = Annotator(
    dataset,
    sampler,
    metadata_path
)

loading configuration file preprocessor_config.json from cache at /home/dani/.cache/huggingface/hub/models--microsoft--resnet-18/snapshots/2f536bd335677c6b111b3d103af458ef57a6145e/preprocessor_config.json
Feature extractor ConvNextFeatureExtractor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ConvNextFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "size": 224
}

PyTorch: setting up devices
loading configuration file config.json from cache at /home/dani/.cache/huggingface/hub/models--microsoft--resnet-18/snapshots/2f536bd335677c6b111b3d103af458ef57a6145e/config.json
Model config ResNetConfig {
  "_name_or_path": "microsoft/resnet-18",
  "architectures": [
    "ResNetForImageClassification"
  ],
  "depths": [
    2,
    2,
    2,
    2
  ],
  "downsample_in_first_stage": false,
  "embedding_size": 64,
  "hidden_act": "relu",
  "hidden_sizes

Epoch,Training Loss,Validation Loss,F1
1,0.593200,0.430091,0.685990
2,0.138900,0.344085,0.721429
3,0.739200,0.315978,0.819444


***** Running Evaluation *****
  Num examples = 39
  Batch size = 10
Saving model checkpoint to resnet-18-finetuned-custom-labeling-project/checkpoint-16
Configuration saved in resnet-18-finetuned-custom-labeling-project/checkpoint-16/config.json
Model weights saved in resnet-18-finetuned-custom-labeling-project/checkpoint-16/pytorch_model.bin
Feature extractor saved in resnet-18-finetuned-custom-labeling-project/checkpoint-16/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 39
  Batch size = 10
Saving model checkpoint to resnet-18-finetuned-custom-labeling-project/checkpoint-32
Configuration saved in resnet-18-finetuned-custom-labeling-project/checkpoint-32/config.json
Model weights saved in resnet-18-finetuned-custom-labeling-project/checkpoint-32/pytorch_model.bin
Feature extractor saved in resnet-18-finetuned-custom-labeling-project/checkpoint-32/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 39
  Batch size = 10
Saving model checkp

UnidentifiedImageError: cannot identify image file '/home/dani/dev/conny-dev/image-classification/data/input/raw/Blendung_Geiger.jpg'

In [35]:
annotator.sampler.model.model

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [28]:
df = pd.DataFrame.from_records(dataset)

In [59]:
y = to_dataset(annotator.unlabeled_data[:100], sampler.labels)

In [51]:
from labeling.model import make_preprocessor, Trainer, collate_pred

In [77]:
sampler.model.predict(y)

***** Running Prediction *****
  Num examples = 100
  Batch size = 10


[[{'label': 'world', 'score': 0.8333007097244263},
  {'label': 'document', 'score': 0.1666993498802185}],
 [{'label': 'world', 'score': 0.7269663214683533},
  {'label': 'document', 'score': 0.27303364872932434}],
 [{'label': 'world', 'score': 0.6427639126777649},
  {'label': 'document', 'score': 0.3572360575199127}],
 [{'label': 'world', 'score': 0.8748810291290283},
  {'label': 'document', 'score': 0.1251189410686493}],
 [{'label': 'world', 'score': 0.7014360427856445},
  {'label': 'document', 'score': 0.29856398701667786}],
 [{'label': 'document', 'score': 0.5753167867660522},
  {'label': 'world', 'score': 0.424683153629303}],
 [{'label': 'world', 'score': 0.6764177083969116},
  {'label': 'document', 'score': 0.323582261800766}],
 [{'label': 'world', 'score': 0.9601656198501587},
  {'label': 'document', 'score': 0.03983433544635773}],
 [{'label': 'world', 'score': 0.7971587181091309},
  {'label': 'document', 'score': 0.20284123718738556}],
 [{'label': 'world', 'score': 0.769630610942

In [56]:
Path('/home/dani/dev/conny-dev/image-classification/data/input/raw/Badfenster.jpg').exists()

True

In [85]:
annotator.sampler = sampler

In [87]:
annotator.sort(annotator.unlabeled_data[:100])

***** Running Prediction *****
  Num examples = 100
  Batch size = 10


[{'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/1709a217-6038-4df0-b6cc-5ce08ce2ff51.jpg'},
  'label': None,
  'score': 0.6929699},
 {'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/1642162753896.jpg'},
  'label': None,
  'score': 0.69287777},
 {'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/20161005_144933.jpg'},
  'label': None,
  'score': 0.69197303},
 {'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/20190516_095451.jpg'},
  'label': None,
  'score': 0.6900464},
 {'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/2.6 Flecken seit 2 Jahren.jpg'},
  'label': None,
  'score': 0.68915284},
 {'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/085223CA-6DB7-4160-B427-3480A66BF2E9.jpeg'},
  'label

In [79]:
sampler(y)

***** Running Prediction *****
  Num examples = 100
  Batch size = 10


(Dataset({
     features: ['image', 'label'],
     num_rows: 100
 }),
 array([0.6929699 , 0.69287777, 0.69197303, 0.6900464 , 0.68915284,
        0.6862478 , 0.6824961 , 0.6824606 , 0.6819408 , 0.68175864,
        0.6801577 , 0.6795973 , 0.677291  , 0.67375994, 0.6734556 ,
        0.67193615, 0.6711677 , 0.66490567, 0.6596555 , 0.65928566,
        0.65629804, 0.656026  , 0.6531657 , 0.65181136, 0.64788085,
        0.6422903 , 0.64139664, 0.64031136, 0.6307365 , 0.6295402 ,
        0.6220398 , 0.6162158 , 0.61558867, 0.61098003, 0.6108281 ,
        0.6096426 , 0.6025416 , 0.59956264, 0.59615684, 0.59417254,
        0.5917546 , 0.5916791 , 0.5862529 , 0.5858738 , 0.58391404,
        0.58243436, 0.5774307 , 0.5632839 , 0.5567051 , 0.5564513 ,
        0.5511859 , 0.5508832 , 0.5433262 , 0.5397223 , 0.5396983 ,
        0.53306067, 0.52290106, 0.5110975 , 0.5043161 , 0.50173753,
        0.49061602, 0.48613572, 0.48557353, 0.47749317, 0.46175915,
        0.46027452, 0.45061374, 0.44836518, 0.

In [80]:
OUT = _

In [84]:
_dataset_to_list(OUT[0])

[{'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/1709a217-6038-4df0-b6cc-5ce08ce2ff51.jpg'},
  'label': None},
 {'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/1642162753896.jpg'},
  'label': None},
 {'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/20161005_144933.jpg'},
  'label': None},
 {'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/20190516_095451.jpg'},
  'label': None},
 {'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/2.6 Flecken seit 2 Jahren.jpg'},
  'label': None},
 {'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/085223CA-6DB7-4160-B427-3480A66BF2E9.jpeg'},
  'label': None},
 {'image': {'bytes': None,
   'path': '/home/dani/dev/conny-dev/image-classification/data/input/raw/196